#### Working codes

In [43]:
from deepface import DeepFace
import cv2
import os
from helper import visulaize_frame, get_id, del_representations

In [37]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
  'fastmtcnn',
]

df_metrics = ["cosine", "euclidean", "euclidean_l2"]

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

In [38]:
def detector(frame, enforce=True):
    try:
        face = DeepFace.extract_faces(frame, detector_backend=backends[0], enforce_detection=enforce)
        return face
    except ValueError:
        return None

In [41]:
from cv2 import VideoCapture

def register_face(id):
    cap = VideoCapture(0)

    while True:
        ret, frame = cap.read()
        face = detector(frame)
        if face is not None:

            # print(face[0]['facial_area'])
            x,y,w,h = face[0]['facial_area'].values()
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
            crop_img = frame[y:y+h, x:x+w]
            cv2.imshow('detect face', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            elif cv2.waitKey(1) & 0xFF == ord('c'):
                # save face
                face_path = "../images/img_db/"+id+".jpg"
                cv2.imwrite(face_path, crop_img)
                print(f" id {id} registered successfully")
                print(f"image stored in {face_path}")
                del_representations()
                
                break
        else:
            h,w = frame.shape[:2]
            text = 'Adjust face and brightness'
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontsize = 1  
            color = (0,255,0)
            thickness = 2
            text_size, _ = cv2.getTextSize(text, font, fontsize, thickness)

            x = (w - text_size[0]) // 2  
            y = (h + text_size[1]) // 2
            cv2.putText(frame, text, (x, y), font, fontsize, color, thickness)
            # cv2.putText(crop_img, 'Align face', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow('detect face', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()    

In [42]:
id = input("Enter id: ")
register_face(id)

 id mne123456 registered successfully
image stored in ../images/img_db/mne123456.jpg


In [46]:
del_representations()

previous representations deleted


In [45]:
def del_representations():
    for obj in os.listdir('../images/img_db/'):
        if obj.startswith('repr'):
            path = os.path.join('../images/img_db/', obj)
            os.remove(path)
    print('previous representations deleted')
    return

In [52]:
DeepFace.find(
    img_path = "../images/img_db/1.jpg",
    db_path = "../images/img_db/",
    model_name = "Facenet512",
    distance_metric = "euclidean_l2",
    enforce_detection = False)

Finding representations: 100%|██████████| 3/3 [00:46<00:00, 15.52s/it]


24-01-15 06:21:54 - Representations stored in ../images/img_db//representations_facenet512.pkl file.Please delete this file when you add new identities in your database.


ValueError: Confirm that ../images/img_db/1.jpg exists

In [60]:
def check_duplicate_registration():
    if 'mme172415.jpg' in os.listdir('./img_db/'):
        return True
    else:
        return False